import sys
!{sys.executable} -m pip install music21
!{sys.executable} -m pip install tqdm



## Methodology:

We write functions to:

1- Encode Each song.

2- Decode each song and save it as midi.

3- Obtain a dict that translates each event into a numerical value.

4- Prepare input sequences and outputs for the three LSTMs to be trained.

5- Train the LSTMs.

6- Use the three trained LSTMs to generate music.

In [ ]:
import music21
import numpy as np
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import pandas as pd
from keras.utils import np_utils
from collections import defaultdict

from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten


In [ ]:
def encode_song(filename):
    """Encodes song in filename. See milestone1 report for encoding"""
    notesRaw=music21.converter.parse(filename)
    notesRaw=notesRaw.flat.notes
        
    pitches=[]
    offsets=[]
        
    for note in notesRaw:
        if isinstance(note,music21.note.Note):
            pitches.append(note.pitch.midi)
            offsets.append(note.offset)
        else:
            chordNotes=[int(b.midi) for b in note.pitches]
            pitches.extend(chordNotes)
            offsets.extend([note.offset]*len(chordNotes))

    pitches=np.array(pitches)
    offsets=np.array(offsets)
    uniqueSortedOffsets=np.sort(np.unique(offsets))
    
    encoding=[]
    
    for i in range(len(uniqueSortedOffsets)-1):
        time=uniqueSortedOffsets[i]
        pitchesHere=pitches[offsets==time]
        goforward='gf'+str(uniqueSortedOffsets[i+1]-time)
        
        encoding.extend(list(np.sort(pitchesHere)))
        encoding.append(goforward)
        
        
    
    return encoding   

In [ ]:
def decode_song(song,filename):
    """Decodes song encoded by encode_song(). song is an encoded song of type list, and filename is
    the path where the new decoded midi file is to reside"""
    stream=[]
    offset=0
    for event in song:
        if 'gf' in str(event):
            offset+=float(event[2:])
        else:
            newNote=music21.note.Note(int(event))
            newNote.offset=offset
            newNote.storedInstrument=music21.instrument.Piano()
            stream.append(newNote)
    midi_stream=music21.stream.Stream(stream)
    midi_stream.write('midi', fp=filename)
    return 1
    
    
    

In [ ]:
#we get the the directories of all the songs in our dataset. 

artists=glob.glob('Music/*') #files are sorted via artist
 
songs=[] #the song filename

for artist in artists:
    songs.extend(glob.glob(artist+'/*'))


In [ ]:
#we encode each song, save them in a dataframe, and then pickle the dataframe.
information=defaultdict(list)

for song in tqdm(songs):
    encoded=encode_song(song)
    _,artist,songname=song.split('\\')
    information['SongName'].append(songname)
    information['Artist'].append(artist)
    information['Encoded'].append(encoded)
    



In [ ]:
DF_to_save=pd.DataFrame.from_dict(information)
DF_to_save.head()

with open('encodings/information.pickle','wb') as file:
    pickle.dump(DF_to_save,file)

In [ ]:
DF=DF_to_save
DF_to_save=[]

In [ ]:
#inWhat follows we create three dictionaries: one for notes and gf (which will be used in the input)
#another for just gfs (output of get_gf), and another for just notes (for get_note)

#we first get all the unique elements of our vocabulary:

vocab=[]
for i in tqdm(range(len(DF))):
    vocab.extend(DF.iloc[i,2])
    vocab=list(set(vocab))
    
notesOnly=[]
gfOnly=[]

for word in vocab:

    if 'gf' in str(word):
        gfOnly.append(word)
    else:
        notesOnly.append(word)

In [ ]:
#we check to see what is the quarter lengths duration of the unique go forward events
numbers=[]
for gf in gfOnly:
    if '/' in gf:
        num,denom=gf.split('/')
        num=num[2:]
        numbers.append(float(num)/float(denom))
    else:
        numbers.append(float(gf[2:]))

In [ ]:
print(sorted(numbers))

From above we see that alot of the unique offsets are essentially the same but differ slightly due to representation by the author of the midi file. We thus have to fix something in the representation of the gfs. This is something we could have fixed in the encode_song function. However to avoid having to open the files again, we ammend the issue in the dataframe.

In [ ]:
np.round(1,4)

In [ ]:
NewColumn=[]
for i in tqdm(range(len(DF))):
    encoding=DF.iloc[i,2]
    #now loop through every word of the encoding
    replacement=[]
    for i,word in enumerate(encoding):
        if 'gf' in str(word):
            #we have a gf, we isolate the word from the letters 'gf'
            keep=word[2:]
            #we now check to see if there is a division symbol:
            if '/' in keep:
                numerator,denominator=keep.split('/')
                keep=np.round(float(numerator)/float(denominator),4)
                replacement.append('gf'+str(keep))
            else:
                replacement.append('gf'+str(np.round(float(keep),4)))
                
        else:
            #its just a note.
            replacement.append(word)
    NewColumn.append(replacement)

In [ ]:
DF['ModedEncoded']=NewColumn


In [ ]:
DF=DF.drop(columns='Encoded')

with open('encodings/information.pickle','wb') as file:
    pickle.dump(DF,file)

In [ ]:
with open('encodings/information.pickle','rb') as file:
    DF=pickle.load(file)

In [ ]:
DF.head()

In [ ]:
DF=DF[DF['Artist']=='chopin']

In [ ]:
DF=DF.iloc[[0,1,2,3],:]

In [ ]:
#we first get all the unique elements of our vocabulary:

vocab=[]
for i in tqdm(range(len(DF))):
    vocab.extend(DF.iloc[i,2])
    vocab=list(set(vocab))
    
notesOnly=[]
gfOnly=[]

for word in vocab:

    if 'gf' in str(word):
        gfOnly.append(word)
    else:
        notesOnly.append(word)

In [ ]:
len(vocab)

In [ ]:
#we check to see what is the quarter lengths duration of the unique go forward events
numbers=[]
for gf in gfOnly:
    if '/' in gf:
        num,denom=gf.split('/')
        num=num[2:]
        numbers.append(float(num)/float(denom))
    else:
        numbers.append(float(gf[2:]))
sorted(numbers)

In [ ]:
#we now can start preparing inputs and outputs for the various neural networks we want to train.

In [1]:
#first step is getting a representation of the vocabulary:
vocab=[]
for i in tqdm(range(len(DF))):
    vocab.extend(DF.iloc[i,2])
    vocab=list(set(vocab))
with open('SavedModels/vocab.pickle','wb') as file:
    pickle.dump(vocab,file)
    
print('The vocabulary has {} elements'.format(str(len(vocab))))

NameError: name 'tqdm' is not defined

In [ ]:
#we create a dict to translate each vocab element to a number and vice versa:
WordToNumber={}
NumberToWord={}

GfToNumber={}
NumberToGf={}

NoteToNumber={}
NumberToNote={}


gf_index=0
note_index=0

for i,word in enumerate(vocab):
    WordToNumber[word]=i
    NumberToWord[i]=word
    if 'gf' in str(word):
        GfToNumber[word]=gf_index
        NumberToGf[gf_index]=word
        gf_index+=1
    else:
        NoteToNumber[word]=note_index
        NumberToNote[note_index]=word
        note_index+=1

with open('SavedModels/WordToNumber.pickle','wb') as file:
    pickle.dump(WordToNumber,file)

with open('SavedModels/NumberToWord.pickle','wb') as file:
    pickle.dump(NumberToWord,file)
    
with open('SavedModels/GfToNumber.pickle','wb') as file:
    pickle.dump(GfToNumber,file)
with open('SavedModels/NumberToGf.pickle','wb') as file:
    pickle.dump(NumberToGf,file)
    
with open('SavedModels/NoteToNumber.pickle','wb') as file:
    pickle.dump(NoteToNumber,file)
    
with open('SavedModels/NumberToNote.pickle','wb') as file:
    pickle.dump(NumberToNote,file)



## LSTM Architecture:

We have 3 neural networks:

1- gf_or_note

2- get_gf

3- get_note



The inputs for all three of the nets are in the same space and have the same representation. The outputs differ. The output of gf_or_note is binary, the output of get_gf is limited to the different available gf's while the output of get_note is limited to the keys on the piano.

We begin by getting a string

In [ ]:
def prepare_sequences_gf_or_note(DF, n_vocab,WordToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            if 'gf' not in str(sequence_in[-1]):
                sequence_out = float('gf' in str(song[i + sequence_length]))
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(sequence_out)
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [ ]:
inp,out=prepare_sequences_gf_or_note(DF,len(WordToNumber),WordToNumber)
print(inp.shape)
print(out.shape)

In [ ]:
def prepare_sequences_get_gf(DF, n_vocab,WordToNumber,GfToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            sequence_out=song[i + sequence_length]
            if 'gf' in str(sequence_out):
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(GfToNumber[sequence_out])
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [ ]:
inp,out=prepare_sequences_get_gf(DF,len(WordToNumber),WordToNumber,GfToNumber)
print(inp.shape)
print(out.shape)

In [ ]:
def prepare_sequences_get_note(DF, n_vocab,WordToNumber,NoteToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            sequence_out=song[i + sequence_length]
            if 'gf' not in str(sequence_out):
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(NoteToNumber[sequence_out])
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [ ]:
inp,out=prepare_sequences_get_note(DF,len(WordToNumber),WordToNumber,NoteToNumber)
print(inp.shape)
print(out.shape)

In [ ]:
def create_network_gf_or_note(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [ ]:
def create_network_get_gf(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [ ]:
def create_network_get_note(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [ ]:
from keras.callbacks import ModelCheckpoint
def train_gf_or_note(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'SavedModels/weights.gf_or_note.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=10000, callbacks=[checkpoint])

In [ ]:
def train_get_gf(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'SavedModels/weights.get_gf.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=10000, callbacks=[checkpoint])

In [ ]:
def train_get_note(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'SavedModels/weights.get_note.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=10000, callbacks=[checkpoint])

In [ ]:
epochs=2000

gf_or_note_input,gf_or_note_output=prepare_sequences_gf_or_note(DF,len(WordToNumber),WordToNumber)
get_gf_input,get_gf_output=prepare_sequences_get_gf(DF,len(WordToNumber),WordToNumber,GfToNumber)
get_note_input,get_note_output=prepare_sequences_get_note(DF,len(WordToNumber),WordToNumber,NoteToNumber)

print('Sequences Prepared, creating models')

gf_or_note=create_network_gf_or_note(gf_or_note_input,gf_or_note_output.shape[1])
get_gf=create_network_get_gf(get_gf_input,get_gf_output.shape[1])
get_note=create_network_get_note(get_note_input,get_note_output.shape[1])

print('Models Created, training in progress')

train_gf_or_note(gf_or_note,gf_or_note_input,gf_or_note_output,epochs)
train_get_gf(get_gf,get_gf_input,get_gf_output,epochs)
train_get_note(get_note,get_note_input,get_note_output,epochs)

print('Training completed')

In [ ]:
#for gf_or_note
index=gf_or_note_input.shape[0]
result=[]
actual=[]
for i in range(index):
    goingin=np.reshape(gf_or_note_input[i,:,:],(1,100,1))
    shouldGoOut=gf_or_note_output[i,:]
    weGet=gf_or_note.predict(goingin)
    weGet=np.argmax(weGet)
    result.append(weGet)
    actual.append(np.argmax(shouldGoOut))
actual=np.array(actual)
result=np.array(result)

print('Accuracy: '+str(np.mean(actual==result)))
#for i in range(len(actual)):
#    if actual[i]==result[i]:
#        print(str(actual[i])+' '+str(result[i]))
#    else:
#        print(str(actual[i])+' '+str(result[i])+'<-------')
        

In [ ]:
#for get_gf
index=get_gf_input.shape[0]
result=[]
actual=[]
for i in range(index):
    goingin=np.reshape(get_gf_input[i,:,:],(1,100,1))
    shouldGoOut=get_gf_output[i,:]
    weGet=get_gf.predict(goingin)
    weGet=np.argmax(weGet)
    result.append(weGet)
    actual.append(np.argmax(shouldGoOut))
actual=np.array(actual)
result=np.array(result)

print('Accuracy: '+str(np.mean(actual==result)))
#for i in range(len(actual)):
#    if actual[i]==result[i]:
#        print(str(actual[i])+' '+str(result[i]))
#    else:
#        print(str(actual[i])+' '+str(result[i])+'<-------')

In [ ]:
#for get_note
index=get_note_input.shape[0]
result=[]
actual=[]
for i in range(index):
    goingin=np.reshape(get_note_input[i,:,:],(1,100,1))
    shouldGoOut=get_note_output[i,:]
    weGet=get_note.predict(goingin)
    weGet=np.argmax(weGet)
    result.append(weGet)
    actual.append(np.argmax(shouldGoOut))
actual=np.array(actual)
result=np.array(result)

print('Accuracy: '+str(np.mean(actual==result)))
#for i in range(len(actual)):
#    if actual[i]==result[i]:
#        print(str(actual[i])+' '+str(result[i]))
#    else:
#        print(str(actual[i])+' '+str(result[i])+'<-------')

In [ ]:
encoding=DF.iloc[0,2]
trialIN=[]
for i in range(100):
    trialIN.append(WordToNumber[encoding[i]])


In [ ]:
encoding

In [ ]:
gf_or_note.predict(np.reshape(trialIN,(1,100,1))/len(vocab))

In [ ]:
WordToNumber.keys()